In [1]:
import numpy as np
import Queue
import time
import math
import random

In [15]:
class NPuzzle(object):
    def __init__(self,n): 
        """assume n takes the form of x^2 - 1 where x is an integer"""
        self.size = n
        self.side = int(math.sqrt(self.size+1))
        self.reset()
        
    def reset(self):
        self.goal = np.array(range(1,self.size+1)+[0])
        self.prevStates = {}
        self.states = Queue.PriorityQueue()
    
    def solve(self, state):
        self.reset();
        self.init = state
        self.prevStates[self.getHash(self.init)] = [self.init,[]]
        
        self.states.put((0,(self.init,[])))
        
        if(not self.isInfeasible(self.init)):
            self.solveHelper()
        else:
            self.printState(self.init)
            print "is infeasible puzzle"
            
    def getHash(self,state):
        key = "".join(str(x) for x in state)
        return key
    
    def isInfeasible(self,state):
        invs = self.getInversionNum(state)
        return invs%2 != 0
    
    def getInversionNum(self,state):
        invs = 0
        for i in range(self.size):
            for j in range(i+1,self.size+1):
                if(state[i] and state[j] and state[i] > state[j]):
                    invs += 1;
        return invs
    
    def getMisplacedNum(self, state):
        # of misplaced entries
        misplaced = 0
        for entry in range(self.size):
            if state[entry] != entry + 1:
                misplaced += 1
        return misplaced
    
    def getManhattanDist(self, state):
        # return total Manhattan distance
        manhattanDistSum = 0
        for x in range(self.side):
            for y in range(self.side):
                value = state[y+x*self.side]
                if value != 0:
                    targetX = (value - 1) / (self.size+1)
                    targetY = (value - 1) % (self.size+1)
                    dx = x - targetX
                    dy = y - targetY
                    manhattanDistSum += abs(dx) + abs(dy)
        return manhattanDistSum
        
    def successor(self,state,moveType):
        zero_pos = np.argmin(np.array(state))
        row = zero_pos/self.side
        col = zero_pos%self.side
        
        # left move
        if(col > 0 and moveType==0):
            temp = state[(col-1)+row*self.side]
            state[(col-1)+row*self.side] = 0
            state[col+row*self.side] = temp
            return True
        
        # right move
        elif(col < (self.side-1) and moveType==1):
            temp = state[(col+1)+row*self.side]
            state[(col+1)+row*self.side] = 0
            state[col+row*self.side] = temp
            return True
        
        # up move
        elif(row > 0 and moveType==2):
            temp = state[(col)+(row-1)*self.side]
            state[col+(row-1)*self.side] = 0
            state[col+row*self.side] = temp
            return True
        
        # down move
        elif(row < (self.side-1) and moveType==3):
            temp = state[(col)+(row+1)*self.side]
            state[col+(row+1)*self.side] = 0
            state[col+row*self.side] = temp
            return True
        
        return False
    
    def isMatched(self, state):
        matched = self.prevStates.get(self.getHash(state)) != None
        return matched
    
    def isSolved(self, state):
        return np.array_equal(np.array(state), self.goal)
    
    def solveHelper(self):
        endLoop = False
        while(not endLoop):
            item = self.states.get() # dequeue
            current = item[1]
            if self.isSolved(current[0]):
                self.printSol(current[1])
                endLoop = True
            else:
                for move in range(4):
                    copy = current[0][:]
                    if(self.successor(copy,move) and not self.isMatched(copy)):
                        moves = current[1][:]
                        moves.append(move)
                        self.prevStates[self.getHash(copy)] = [copy,moves]
                        priority = self.getManhattanDist(copy) + len(moves) # A*
                        self.states.put((priority,(copy,moves))) # enqueue
                        if(self.isSolved(copy)):
                            self.printSol(moves)
                            endLoop = True
        
    def printState(self,state):
        for pos in range(self.size+1):
            print str(state[pos]) + " ",
            if (pos+1)%self.side == 0: 
                print
        
    def printMove(self,move):
        if(move==0):
            print "Swap Left"
        elif(move==1):
            print "Swap Right"
        elif(move==2):
            print "Swap UP"
        elif(move==3):
            print "Swap Down"
            
    def printSol(self,moves):
        print "Length of solution: %d" %(len(moves))
        state = self.init[:]
        self.printState(state)
        for move in moves:
            self.printMove(move)
            self.successor(state,move)
            self.printState(state)
            
    def randState(self):
        return random.sample(range(self.size+1), self.size+1)

In [6]:
solver = NPuzzle(8)
solver.solve([8,1,2,0,4,3,7,6,5])
solver.solve([2,8,3,1,6,4,7,0,5])
solver.solve([1,2,3,4,0,5,7,8,6])

8  1  2 
0  4  3 
7  6  5 
is infeasible puzzle
2  8  3 
1  6  4 
7  0  5 
is infeasible puzzle
Length of solution: 2
1  2  3 
4  0  5 
7  8  6 
Swap Right
1  2  3 
4  5  0 
7  8  6 
Swap Down
1  2  3 
4  5  6 
7  8  0 


In [16]:
start = time.time()
solver.solve([1,8,7,2,0,6,3,4,5])
end = time.time()
print end - start

Length of solution: 24
1  8  7 
2  0  6 
3  4  5 
Swap UP
1  0  7 
2  8  6 
3  4  5 
Swap Right
1  7  0 
2  8  6 
3  4  5 
Swap Down
1  7  6 
2  8  0 
3  4  5 
Swap Left
1  7  6 
2  0  8 
3  4  5 
Swap UP
1  0  6 
2  7  8 
3  4  5 
Swap Left
0  1  6 
2  7  8 
3  4  5 
Swap Down
2  1  6 
0  7  8 
3  4  5 
Swap Down
2  1  6 
3  7  8 
0  4  5 
Swap Right
2  1  6 
3  7  8 
4  0  5 
Swap UP
2  1  6 
3  0  8 
4  7  5 
Swap Left
2  1  6 
0  3  8 
4  7  5 
Swap UP
0  1  6 
2  3  8 
4  7  5 
Swap Right
1  0  6 
2  3  8 
4  7  5 
Swap Down
1  3  6 
2  0  8 
4  7  5 
Swap Left
1  3  6 
0  2  8 
4  7  5 
Swap Down
1  3  6 
4  2  8 
0  7  5 
Swap Right
1  3  6 
4  2  8 
7  0  5 
Swap Right
1  3  6 
4  2  8 
7  5  0 
Swap UP
1  3  6 
4  2  0 
7  5  8 
Swap UP
1  3  0 
4  2  6 
7  5  8 
Swap Left
1  0  3 
4  2  6 
7  5  8 
Swap Down
1  2  3 
4  0  6 
7  5  8 
Swap Down
1  2  3 
4  5  6 
7  0  8 
Swap Right
1  2  3 
4  5  6 
7  8  0 
1.0000679493
